<h1><center>MPST: A Corpus of Movie Plot Synopses with Tags</center></h1>

In [1]:
!pip install scikit-multilearn

import re
import os
import tqdm
import nltk
import pickle
import sqlite3
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import xgboost as xgb
import tensorflow as tf
from sklearn import metrics
from tensorflow import keras
from nltk.corpus import words
from datetime import datetime
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from itertools import combinations
from keras.models import load_model
from keras.models import Sequential
from tensorflow.keras import layers
from nltk.stem import SnowballStemmer
from sklearn.pipeline import Pipeline
from nltk.tokenize import sent_tokenize
from keras.preprocessing import sequence
from scipy.sparse import coo_matrix, hstack
from tensorflow.keras.utils import plot_model
from keras.layers.embeddings import Embedding
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score,precision_score,recall_score,hamming_loss
from keras.layers import Conv1D, Conv2D, Dense, Dropout, Flatten, LSTM, GlobalMaxPooling1D, MaxPooling2D, Activation, BatchNormalization

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')
warnings.filterwarnings("ignore")
stemmer = SnowballStemmer('english')

%autosave 120

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

Autosaving every 120 seconds


In [2]:
data_with_all_tags = pd.read_csv("data_with_all_tags.csv")
data_with_all_tags.head()

,title,plot_synopsis,tags,split,synopsis_source,cnt_dup,tag_count,synopsis_count,synopsis_sent_count,CleanedSynopsis,...,sentimental,storytelling,stupid,suicidal,suspenseful,thought-provoking,tragedy,violence,western,whimsical
0,$,"Set in Hamburg, West Germany, several criminal...",murder,test,imdb,1,1,648,26,set hamburg west germani sever crimin take adv...,...,0,0,0,0,0,0,0,0,0,0
1,$windle,A 6th grader named Griffin Bing decides to gat...,flashback,train,wikipedia,1,1,353,14,grader name griffin bing decid gather entir gr...,...,0,0,0,0,0,0,0,0,0,0
2,'71,"Gary Hook, a new recruit to the British Army, ...","suspenseful, neo noir, murder, violence",train,wikipedia,1,4,699,39,gari hook new recruit british armi take leav m...,...,0,0,0,0,1,0,0,1,0,0
3,'A' gai wak,Sergeant Dragon Ma (Jackie Chan) is part of th...,"cult, violence",train,wikipedia,1,2,665,41,sergeant dragon jacki chan part hong kong mari...,...,0,0,0,0,0,0,0,1,0,0
4,'Breaker' Morant,"In Pretoria, South Africa, in 1902, Major Char...","murder, anti war, violence, flashback, tragedy...",train,wikipedia,1,6,1694,140,pretoria south africa major charl bolton rod m...,...,0,0,0,0,0,0,1,1,0,0


In [3]:
conn = sqlite3.connect('data.db')
data_with_all_tags.to_sql('data', conn, if_exists='replace', index=False)
train = pd.read_sql("Select * From data where split = 'train' OR split='val'",conn)
test =  pd.read_sql("Select * From data where split = 'test'",conn)
conn.close()

In [4]:
X_train = train["CleanedSynopsis"]
y_train= train["tags"]

X_test = test["CleanedSynopsis"]
y_test= test["tags"]

In [5]:
def tokenize(x):
    x=x.split(',')
    tags=[i.strip() for i in x] #Some tags contains whitespaces before them, so we need to strip them
    return tags

cnt_vectorizer = CountVectorizer(tokenizer = tokenize, max_features=5, binary='true').fit(y_train)
y_train_multilabel = cnt_vectorizer.transform(y_train)
y_test_multilabel = cnt_vectorizer.transform(y_test)

<h1> 1. TfidfVectorizer with 1 grams: </h1>

In [6]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,1))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 657) Y : (10989, 5)
Dimensions of test data X: (2768, 657) Y: (2768, 5)


<h2> 1.1 OneVsRestClassifier + MultinomialNB:</h2>

In [7]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [8]:
prediction1 = clf.predict(X_test_multilabel)

precision1 = precision_score(y_test_multilabel, prediction1, average='micro')

recall1 = recall_score(y_test_multilabel, prediction1, average='micro')

f1_score1 = 2*((precision1 * recall1)/(precision1 + recall1))

print("precision1: {:.4f}, recall1: {:.4f}, F1-measure: {:.4f}".format(precision1, recall1, f1_score1))

precision1: 0.4110, recall1: 0.6757, F1-measure: 0.5111


In [9]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction1[k])[0],"\n")

Movie:  Nuovo Cinema Paradiso
Actual genre:  melodrama, cute, sentimental, flashback
Predicted tag:  ['romantic'] 

Movie:  Blackhat
Actual genre:  suspenseful, neo noir, murder
Predicted tag:  ['cult' 'murder' 'violence'] 

Movie:  Out for Blood
Actual genre:  flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Carnal Knowledge
Actual genre:  comedy
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Unlawful Entry
Actual genre:  insanity, suspenseful, neo noir, murder
Predicted tag:  ['cult' 'murder' 'violence'] 



<h2> 1.2 OneVsRestClassifier + SGDClassifier with LOG Loss:</h2>

In [10]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [11]:
prediction2 = clf.predict(X_test_multilabel)

precision2 = precision_score(y_test_multilabel, prediction2, average='micro')

recall2 = recall_score(y_test_multilabel, prediction2, average='micro')

f1_score2 = 2*((precision2 * recall2)/(precision2 + recall2))

print("precision2: {:.4f}, recall2: {:.4f}, F1-measure: {:.4f}".format(precision2, recall2, f1_score2))

precision2: 0.4477, recall2: 0.6743, F1-measure: 0.5381


In [12]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction2[k])[0],"\n")

Movie:  High Wall
Actual genre:  murder
Predicted tag:  ['flashback' 'murder'] 

Movie:  Feng hou
Actual genre:  cult, grindhouse film
Predicted tag:  ['violence'] 

Movie:  Zerophilia
Actual genre:  pornographic
Predicted tag:  [] 

Movie:  Deadline at Dawn
Actual genre:  mystery, murder
Predicted tag:  ['murder' 'romantic'] 

Movie:  Assault
Actual genre:  murder
Predicted tag:  [] 



<h2> 1.3 OneVsRestClassifier + SGDClassifier with Hinge Loss:</h2>

In [13]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [14]:
prediction3 = clf.predict(X_test_multilabel)

precision3 = precision_score(y_test_multilabel, prediction3, average='micro')

recall3 = recall_score(y_test_multilabel, prediction3, average='micro')

f1_score3 = 2*((precision3 * recall3)/(precision3 + recall3))

print("precision3: {:.4f}, recall3: {:.4f}, F1-measure: {:.4f}".format(precision3, recall3, f1_score3))

precision3: 0.4320, recall3: 0.6684, F1-measure: 0.5248


In [15]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction3[k])[0],"\n")

Movie:  Aan
Actual genre:  romantic
Predicted tag:  ['romantic'] 

Movie:  Aan
Actual genre:  romantic
Predicted tag:  ['romantic'] 

Movie:  Evita
Actual genre:  avant garde, boring, murder, dramatic, cult, historical
Predicted tag:  ['romantic'] 

Movie:  Men of War
Actual genre:  revenge, violence
Predicted tag:  ['murder' 'violence'] 

Movie:  The Hours
Actual genre:  tragedy, dramatic, queer
Predicted tag:  ['flashback' 'romantic'] 



<h2> 1.4  OneVsRestClassifier + LogisticRegression:</h2>

In [16]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [17]:
prediction4 = clf.predict(X_test_multilabel)

precision4 = precision_score(y_test_multilabel, prediction4, average='micro')

recall4 = recall_score(y_test_multilabel, prediction4, average='micro')

f1_score4 = 2*((precision4 * recall4)/(precision4 + recall4))

print("precision4: {:.4f}, recall4: {:.4f}, F1-measure: {:.4f}".format(precision4, recall4, f1_score4))

precision4: 0.4469, recall4: 0.6679, F1-measure: 0.5355


In [18]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction4[k])[0],"\n")

Movie:  Kaboom
Actual genre:  psychedelic, queer
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Nim's Island
Actual genre:  flashback, absurd, psychedelic, storytelling, home movie
Predicted tag:  ['flashback'] 

Movie:  Babel
Actual genre:  dramatic, murder, flashback
Predicted tag:  ['flashback' 'murder'] 

Movie:  Charlie Wilson's War
Actual genre:  satire, entertaining, historical
Predicted tag:  [] 

Movie:  No Strings Attached
Actual genre:  boring, adult comedy, cute, flashback, romantic, entertaining
Predicted tag:  ['flashback' 'romantic'] 



<h1>2. TfidfVectorizer with (1 - 2 Grams):<?h1>

In [19]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,2))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 5)
Dimensions of test data X: (2768, 666) Y: (2768, 5)


<H2> 2.1 OneVsRestClassifier + MultinomialNB :</H2>

In [20]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [21]:
prediction5 = clf.predict(X_test_multilabel)

precision5 = precision_score(y_test_multilabel, prediction5, average='micro')

recall5 = recall_score(y_test_multilabel, prediction5, average='micro')

f1_score5 = 2*((precision5 * recall5)/(precision5 + recall5))

print("precision5: {:.4f}, recall5: {:.4f}, F1-measure: {:.4f}".format(precision5, recall5, f1_score5))

precision5: 0.4120, recall5: 0.6762, F1-measure: 0.5120


In [22]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction5[k])[0],"\n")

Movie:  Driven to Kill
Actual genre:  revenge, murder, violence
Predicted tag:  ['murder' 'romantic'] 

Movie:  Unspeakable
Actual genre:  paranormal, violence, murder, sadist, flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Talladega Nights: The Ballad of Ricky Bobby
Actual genre:  boring, entertaining, adult comedy, comic
Predicted tag:  ['cult'] 

Movie:  Konga
Actual genre:  murder
Predicted tag:  ['cult' 'flashback' 'romantic'] 

Movie:  Chaos
Actual genre:  suspenseful, neo noir, murder, violence, flashback, action, revenge
Predicted tag:  ['murder' 'violence'] 



<h2> 2.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [23]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [24]:
prediction6 = clf.predict(X_test_multilabel)

precision6 = precision_score(y_test_multilabel, prediction6, average='micro')

recall6 = recall_score(y_test_multilabel, prediction6, average='micro')

f1_score6 = 2*((precision6 * recall6)/(precision6 + recall6))

print("precision6: {:.4f}, recall6: {:.4f}, F1-measure: {:.4f}".format(precision6, recall6, f1_score6))

precision6: 0.4438, recall6: 0.6827, F1-measure: 0.5379


In [25]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction6[k])[0],"\n")

Movie:  Scooby-Doo Goes Hollywood
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Mr. & Mrs. Smith
Actual genre:  comedy
Predicted tag:  ['romantic'] 

Movie:  Angry Video Game Nerd: The Movie
Actual genre:  violence
Predicted tag:  ['flashback'] 

Movie:  God Bless America
Actual genre:  violence, comedy, satire, murder
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Elephant
Actual genre:  mystery, cruelty, murder, bleak, violence, revenge, sadist
Predicted tag:  ['cult' 'murder' 'violence'] 



<h2> 2.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [26]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [27]:
prediction7 = clf.predict(X_test_multilabel)

precision7 = precision_score(y_test_multilabel, prediction7, average='micro')

recall7 = recall_score(y_test_multilabel, prediction7, average='micro')

f1_score7 = 2*((precision7 * recall7)/(precision7 + recall7))

print("precision7: {:.4f}, recall7: {:.4f}, F1-measure: {:.4f}".format(precision7, recall7, f1_score7))

precision7: 0.4287, recall7: 0.6818, F1-measure: 0.5264


In [28]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction7[k])[0],"\n")

Movie:  Kenka erejî
Actual genre:  satire, comic
Predicted tag:  ['romantic'] 

Movie:  Lung Fu Moon
Actual genre:  flashback
Predicted tag:  ['cult' 'violence'] 

Movie:  The Descendants
Actual genre:  boring, dramatic, plot twist, romantic, entertaining, sentimental
Predicted tag:  ['romantic'] 

Movie:  La piscine
Actual genre:  dramatic, murder
Predicted tag:  ['romantic'] 

Movie:  The Baby of Mâcon
Actual genre:  comedy, avant garde, cruelty, cult, violence, psychedelic, satire
Predicted tag:  [] 



<h2> 2.4 OneVsRestClassifier + LogisticRegression:</h2>

In [29]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [30]:
prediction8 = clf.predict(X_test_multilabel)

precision8 = precision_score(y_test_multilabel, prediction8, average='micro')

recall8 = recall_score(y_test_multilabel, prediction8, average='micro')

f1_score8 = 2*((precision8 * recall8)/(precision8 + recall8))

print("precision8: {:.4f}, recall8: {:.4f}, F1-measure: {:.4f}".format(precision8, recall8, f1_score8))

precision8: 0.4469, recall8: 0.6676, F1-measure: 0.5354


In [31]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction8[k])[0],"\n")

Movie:  Cursed
Actual genre:  comedy, suspenseful, murder, cult, violence, absurd, psychedelic, satire, revenge
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Blues in the Night
Actual genre:  revenge, murder, violence
Predicted tag:  ['romantic'] 

Movie:  Cousins
Actual genre:  revenge, romantic
Predicted tag:  ['romantic'] 

Movie:  Flight of Fury
Actual genre:  violence
Predicted tag:  ['murder' 'violence'] 

Movie:  Astérix et les Vikings
Actual genre:  psychedelic
Predicted tag:  [] 



<h1>3. TfidfVectorizer with (1 - 3 Grams):<?h1>

In [32]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1,3))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 5)
Dimensions of test data X: (2768, 666) Y: (2768, 5)


<H2> 3.1 OneVsRestClassifier + MultinomialNB :</H2>

In [33]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [34]:
prediction9 = clf.predict(X_test_multilabel)

precision9 = precision_score(y_test_multilabel, prediction9, average='micro')

recall9 = recall_score(y_test_multilabel, prediction9, average='micro')

f1_score9 = 2*((precision9 * recall9)/(precision9 + recall9))

print("precision9: {:.4f}, recall9: {:.4f}, F1-measure: {:.4f}".format(precision9, recall9, f1_score9))

precision9: 0.4120, recall9: 0.6762, F1-measure: 0.5120


In [35]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction9[k])[0],"\n")

Movie:  Nothing But Trouble
Actual genre:  comedy, cult
Predicted tag:  ['violence'] 

Movie:  Dragonfly
Actual genre:  comedy, mystery, paranormal, flashback, romantic, melodrama, tragedy, suspenseful
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Roadside Romeo
Actual genre:  romantic
Predicted tag:  ['romantic'] 

Movie:  Memento
Actual genre:  dark, avant garde, suspenseful, murder, neo noir, boring, bleak, realism, cult, violence, psychological, plot twist, flashback, mystery, revenge, storytelling
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  How to Steal a Million
Actual genre:  comedy
Predicted tag:  ['cult' 'flashback' 'murder'] 



<H2> 3.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [36]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [37]:
prediction10 = clf.predict(X_test_multilabel)

precision10 = precision_score(y_test_multilabel, prediction10, average='micro')

recall10 = recall_score(y_test_multilabel, prediction10, average='micro')

f1_score10 = 2*((precision10 * recall10)/(precision10 + recall10))

print("precision10: {:.4f}, recall10: {:.4f}, F1-measure: {:.4f}".format(precision10, recall10, f1_score10))

precision10: 0.4495, recall10: 0.6576, F1-measure: 0.5340


In [38]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction10[k])[0],"\n")

Movie:  Humpty Dumpty
Actual genre:  good versus evil, psychedelic
Predicted tag:  ['romantic' 'violence'] 

Movie:  Bill & Ted's Excellent Adventure
Actual genre:  comedy, cult, historical fiction, alternate history, historical, entertaining
Predicted tag:  ['cult' 'romantic'] 

Movie:  L'ultima orgia del III Reich
Actual genre:  violence, cruelty, murder, sadist, flashback
Predicted tag:  ['cult' 'romantic' 'violence'] 

Movie:  The Skeleton Key
Actual genre:  suspenseful, psychedelic, gothic, flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  White Tiger
Actual genre:  murder
Predicted tag:  [] 



<h2> 3.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [39]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [40]:
prediction11 = clf.predict(X_test_multilabel)

precision11 = precision_score(y_test_multilabel, prediction11, average='micro')

recall11 = recall_score(y_test_multilabel, prediction11, average='micro')

f1_score11 = 2*((precision11 * recall11)/(precision11 + recall11))

print("precision11: {:.4f}, recall11: {:.4f}, F1-measure: {:.4f}".format(precision11, recall11, f1_score11))

precision11: 0.4351, recall11: 0.6762, F1-measure: 0.5295


In [41]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction11[k])[0],"\n")

Movie:  Teacher's Pet
Actual genre:  romantic, murder
Predicted tag:  [] 

Movie:  Living Hell
Actual genre:  suspenseful, violence
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Victory at Entebbe
Actual genre:  violence
Predicted tag:  ['cult' 'violence'] 

Movie:  Jungledyret
Actual genre:  insanity, psychedelic
Predicted tag:  ['romantic'] 

Movie:  Let Us Prey
Actual genre:  violence, murder, flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 



<h2> 3.4 OneVsRestClassifier + LogisticRegression:</h2>

In [42]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [43]:
prediction12 = clf.predict(X_test_multilabel)

precision12 = precision_score(y_test_multilabel, prediction12, average='micro')

recall12 = recall_score(y_test_multilabel, prediction12, average='micro')

f1_score12 = 2*((precision12 * recall12)/(precision12 + recall12))

print("precision12: {:.4f}, recall12: {:.4f}, F1-measure: {:.4f}".format(precision12, recall12, f1_score12))

precision12: 0.4469, recall12: 0.6676, F1-measure: 0.5354


In [44]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction12[k])[0],"\n")

Movie:  Cheyenne Wildcat
Actual genre:  murder
Predicted tag:  ['flashback' 'murder' 'violence'] 

Movie:  Moonlight
Actual genre:  revenge
Predicted tag:  ['cult' 'flashback' 'romantic'] 

Movie:  Ben-Hur: A Tale of the Christ
Actual genre:  revenge
Predicted tag:  ['flashback'] 

Movie:  I Me Wed
Actual genre:  romantic
Predicted tag:  ['romantic'] 

Movie:  Paycheck
Actual genre:  neo noir, murder, revenge, flashback, romantic, suspenseful, sci-fi
Predicted tag:  ['cult' 'flashback'] 



<h1>4. TfidfVectorizer with (1 - 4 Grams):</h1>

In [45]:
tf_vectorizer = TfidfVectorizer(min_df=0.09, tokenizer = lambda x: x.split(" "), ngram_range=(1, 4))

X_train_multilabel = tf_vectorizer.fit_transform(X_train)
X_test_multilabel = tf_vectorizer.transform(X_test)

print("Dimensions of train data X:",X_train_multilabel.shape, "Y :",y_train_multilabel.shape)
print("Dimensions of test data X:",X_test_multilabel.shape,"Y:",y_test_multilabel.shape)

Dimensions of train data X: (10989, 666) Y : (10989, 5)
Dimensions of test data X: (2768, 666) Y: (2768, 5)


<H2> 4.1 OneVsRestClassifier + MultinomialNB :</H2>

In [46]:
mb = MultinomialNB(class_prior = [0.5, 0.5])

clf = OneVsRestClassifier(mb)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=MultinomialNB(alpha=1.0, class_prior=[0.5, 0.5],
                                            fit_prior=True),
                    n_jobs=None)

In [47]:
prediction13 = clf.predict(X_test_multilabel)

precision13 = precision_score(y_test_multilabel, prediction13, average='micro')

recall13 = recall_score(y_test_multilabel, prediction13, average='micro')

f1_score13 = 2*((precision13 * recall13)/(precision13 + recall13))

print("precision13: {:.4f}, recall13: {:.4f}, F1-measure: {:.4f}".format(precision13, recall13, f1_score13))

precision13: 0.4120, recall13: 0.6762, F1-measure: 0.5120


In [48]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction13[k])[0],"\n")

Movie:  The Chronicles of Narnia: Prince Caspian
Actual genre:  good versus evil, violence, fantasy, boring, romantic
Predicted tag:  ['cult' 'flashback' 'violence'] 

Movie:  Jawbreaker
Actual genre:  comedy, cruelty, murder, cult, satire, prank
Predicted tag:  ['romantic'] 

Movie:  Angry Video Game Nerd: The Movie
Actual genre:  violence
Predicted tag:  ['cult' 'violence'] 

Movie:  Fluke
Actual genre:  cruelty, flashback
Predicted tag:  ['flashback'] 

Movie:  Vals Im Bashir
Actual genre:  psychological, thought-provoking, flashback, psychedelic, autobiographical, historical, storytelling, sentimental
Predicted tag:  ['flashback' 'violence'] 



<h2> 4.2 OneVsRestClassifier + SGDClassifier with LOG Loss :</h2>

In [49]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [50]:
prediction14 = clf.predict(X_test_multilabel)

precision14 = precision_score(y_test_multilabel, prediction14, average='micro')

recall14 = recall_score(y_test_multilabel, prediction14, average='micro')

f1_score14 = 2*((precision14 * recall14)/(precision14 + recall14))

print("precision14: {:.4f}, recall14: {:.4f}, F1-measure: {:.4f}".format(precision14, recall14, f1_score14))

precision14: 0.4374, recall14: 0.6835, F1-measure: 0.5335


In [51]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction14[k])[0],"\n")

Movie:  Zeher
Actual genre:  romantic
Predicted tag:  ['flashback' 'murder'] 

Movie:  Capturing the Friedmans
Actual genre:  brainwashing, romantic, home movie
Predicted tag:  ['murder'] 

Movie:  Vanishing Point
Actual genre:  comedy, neo noir, cult, flashback, insanity, revenge
Predicted tag:  ['cult' 'murder' 'violence'] 

Movie:  Zoombies
Actual genre:  violence
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Waist Deep
Actual genre:  blaxploitation, revenge, neo noir, murder, violence
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 



<h2> 4.3 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [52]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [53]:
prediction15 = clf.predict(X_test_multilabel)

precision15 = precision_score(y_test_multilabel, prediction15, average='micro')

recall15 = recall_score(y_test_multilabel, prediction15, average='micro')

f1_score15 = 2*((precision15 * recall15)/(precision15 + recall15))

print("precision15: {:.4f}, recall15: {:.4f}, F1-measure: {:.4f}".format(precision15, recall15, f1_score15))

precision15: 0.4344, recall15: 0.6832, F1-measure: 0.5311


In [54]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction15[k])[0],"\n")

Movie:  Chi sei?
Actual genre:  paranormal, psychedelic
Predicted tag:  ['cult' 'flashback' 'murder'] 

Movie:  Critters 3
Actual genre:  flashback
Predicted tag:  ['violence'] 

Movie:  Blitz Wolf
Actual genre:  psychedelic
Predicted tag:  [] 

Movie:  Das Wachsfigurenkabinett
Actual genre:  good versus evil, insanity
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Last Summer
Actual genre:  violence, cruelty, romantic
Predicted tag:  ['romantic'] 



<h2> 4.4 OneVsRestClassifier + LogisticRegression:</h2>

In [55]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [56]:
prediction16 = clf.predict(X_test_multilabel)

precision16 = precision_score(y_test_multilabel, prediction16, average='micro')

recall16 = recall_score(y_test_multilabel, prediction16, average='micro')

f1_score16 = 2*((precision16 * recall16)/(precision16 + recall16))

print("precision16: {:.4f}, recall16: {:.4f}, F1-measure: {:.4f}".format(precision16, recall16, f1_score16))

precision16: 0.4469, recall16: 0.6676, F1-measure: 0.5354


In [57]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction16[k])[0],"\n")

Movie:  Dark Victory
Actual genre:  tragedy, romantic, melodrama
Predicted tag:  ['flashback' 'romantic'] 

Movie:  Star Wars: The Force Unleashed
Actual genre:  violence
Predicted tag:  ['cult' 'murder' 'violence'] 

Movie:  Cinderella Swings It
Actual genre:  philosophical
Predicted tag:  [] 

Movie:  Wide Sargasso Sea
Actual genre:  insanity
Predicted tag:  ['flashback' 'murder' 'romantic'] 

Movie:  Let's Scare Jessica to Death
Actual genre:  insanity, cult, psychedelic, murder
Predicted tag:  ['flashback' 'murder' 'romantic'] 



<h1> Conclusion: </h1>

In [58]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model','Vectorizer','ngrams','Precision','recall','f1_score']



tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 1)', round(precision1, 3),round(recall1, 3), round(f1_score1, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 1)', round(precision2, 3), round(recall2, 3), round(f1_score2, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 1)' ,round(precision3, 3),  round(recall3, 3), round(f1_score3, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 1)', round(precision4, 3), round(recall4, 3), round(f1_score4, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 2)', round(precision5, 3), round(recall5, 3),  round(f1_score5, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 2)', round(precision6, 3),  round(recall6, 3),  round(f1_score6, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 2)', round(precision7, 3),  round(recall7, 3),  round(f1_score7, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 2)', round(precision8, 3),  round(recall8, 3),  round(f1_score8, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])


tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 3)', round(precision9, 3),  round(recall9, 3), round(f1_score9, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 3)', round(precision10, 3),  round(recall10, 3), round(f1_score10, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 3)', round(precision11, 3),  round(recall11, 3), round(f1_score11, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 3)', round(precision12, 3),  round(recall12, 3), round(f1_score12, 3)])

tabel.add_row(['','','','','',''])
tabel.add_row(['','','','','',''])

tabel.add_row(['MultinomialNB', 'TfidfVectorizer', '(1, 4)', round(precision13, 3), round(recall13, 3), round(f1_score13, 3)])

tabel.add_row(['SGDClassifier(log)', 'TfidfVectorizer', '(1, 4)', round(precision14, 3), round(recall14, 3), round(f1_score14, 3)])

tabel.add_row(['SGDClassifier(hinge)','TfidfVectorizer','(1, 4)', round(precision15, 3), round(recall15, 3), round(f1_score15, 3)])

tabel.add_row(['LogisticRegression','TfidfVectorizer','(1, 4)', round(precision16, 3),  round(recall16, 3),  round(f1_score16, 3)])



print(tabel)

+----------------------+-----------------+--------+-----------+--------+----------+
|        Model         |    Vectorizer   | ngrams | Precision | recall | f1_score |
+----------------------+-----------------+--------+-----------+--------+----------+
|    MultinomialNB     | TfidfVectorizer | (1, 1) |   0.411   | 0.676  |  0.511   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 1) |   0.448   | 0.674  |  0.538   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 1) |   0.432   | 0.668  |  0.525   |
|  LogisticRegression  | TfidfVectorizer | (1, 1) |   0.447   | 0.668  |  0.535   |
|                      |                 |        |           |        |          |
|                      |                 |        |           |        |          |
|    MultinomialNB     | TfidfVectorizer | (1, 2) |   0.412   | 0.676  |  0.512   |
|  SGDClassifier(log)  | TfidfVectorizer | (1, 2) |   0.444   | 0.683  |  0.538   |
| SGDClassifier(hinge) | TfidfVectorizer | (1, 2) |   0.429   | 0.682  |  0.

<h1>5. Word2Vec</h1>

In [59]:
X_train_new = []

for i in tqdm(range(len(list(X_train)))):
    X_train_new.append(X_train[i].split(" "))

100%|█████████████████████████████████████████████████████████████████████████| 10989/10989 [00:00<00:00, 17291.14it/s]


In [60]:
with open('glove.6B.300d.pkl', 'rb') as f:
    new_model = pickle.load(f)
    words =  set(new_model.keys())

In [61]:
X_train_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_train.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_train_multilabel.append(vector)

100%|██████████████████████████████████████████████████████████████████████████| 10989/10989 [00:09<00:00, 1146.73it/s]


In [62]:
X_test_multilabel = []; # the avg-w2v for each sentence/review is stored in this list
for sentence in tqdm(X_test.values): # for each review/sentence
    vector = np.zeros(300) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sentence.split():
        if word in words:
            vector += new_model[word]
            cnt_words += 1
    if cnt_words != 0:
        vector /= cnt_words
    X_test_multilabel.append(vector)

100%|████████████████████████████████████████████████████████████████████████████| 2768/2768 [00:02<00:00, 1099.68it/s]


<H2> 5.1 OneVsRestClassifier + SGDClassifier with LOG Loss :</H2>

In [63]:
sgl = SGDClassifier(loss='log', class_weight='balanced')

clf = OneVsRestClassifier(sgl)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal', loss='log',
                                            max_iter=1000, n_iter_no_change=5,
                                            n_jobs=None, penalty='l2',
                                            power_t=0.5, random_state=None,
                                            shuffle=True, tol=0.001,
                                            validation_fraction=0.1, verbose=0,
                                            warm_start=False),
                    n_jobs=None)

In [64]:
prediction17 = clf.predict(X_test_multilabel)

precision17 = precision_score(y_test_multilabel, prediction17, average='micro')

recall17 = recall_score(y_test_multilabel, prediction17, average='micro')

f1_score17 = 2*((precision17 * recall17)/(precision17 + recall17))

print("precision17: {:.4f}, recall17: {:.4f}, F1-measure: {:.4f}".format(precision17, recall17, f1_score17))

precision17: 0.4035, recall17: 0.6539, F1-measure: 0.4990


In [65]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction17[k])[0],"\n")

Movie:  The American Ruling Class
Actual genre:  satire
Predicted tag:  ['flashback'] 

Movie:  The Perfect Host
Actual genre:  plot twist, psychedelic, flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 

Movie:  Thunderheart
Actual genre:  suspenseful, murder, flashback
Predicted tag:  ['murder' 'violence'] 

Movie:  The Creeping Flesh
Actual genre:  murder
Predicted tag:  ['flashback' 'violence'] 

Movie:  Botched
Actual genre:  comedy, violence
Predicted tag:  ['cult' 'flashback' 'murder' 'violence'] 



<h2> 5.2 OneVsRestClassifier + SGDClassifier with HINGE Loss : </h2>

In [66]:
sgh = SGDClassifier(loss='hinge', class_weight='balanced')

clf = OneVsRestClassifier(sgh)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=SGDClassifier(alpha=0.0001, average=False,
                                            class_weight='balanced',
                                            early_stopping=False, epsilon=0.1,
                                            eta0=0.0, fit_intercept=True,
                                            l1_ratio=0.15,
                                            learning_rate='optimal',
                                            loss='hinge', max_iter=1000,
                                            n_iter_no_change=5, n_jobs=None,
                                            penalty='l2', power_t=0.5,
                                            random_state=None, shuffle=True,
                                            tol=0.001, validation_fraction=0.1,
                                            verbose=0, warm_start=False),
                    n_jobs=None)

In [67]:
prediction18 = clf.predict(X_test_multilabel)

precision18 = precision_score(y_test_multilabel, prediction18, average='micro')

recall18 = recall_score(y_test_multilabel, prediction18, average='micro')

f1_score18 = 2*((precision18 * recall18)/(precision18 + recall18))

print("precision18: {:.4f}, recall18: {:.4f}, F1-measure: {:.4f}".format(precision18, recall18, f1_score18))

precision18: 0.4360, recall18: 0.6456, F1-measure: 0.5205


In [68]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction18[k])[0],"\n")

Movie:  The Ghost Writer
Actual genre:  boring, mystery, murder, dramatic, intrigue, atmospheric, suspenseful
Predicted tag:  ['flashback' 'murder' 'romantic'] 

Movie:  See No Evil, Hear No Evil
Actual genre:  comedy, murder, adult comedy
Predicted tag:  ['cult' 'murder'] 

Movie:  The Joneses
Actual genre:  satire
Predicted tag:  ['romantic'] 

Movie:  Milk
Actual genre:  avant garde, murder, dramatic, violence, thought-provoking, flashback, historical, sentimental
Predicted tag:  [] 

Movie:  Chernobyl Diaries
Actual genre:  violence, dark, cruelty, murder
Predicted tag:  ['murder' 'violence'] 



<h2> 5.3 OneVsRestClassifier + LogisticRegression:</h2>

In [69]:
lr = LogisticRegression(class_weight='balanced')

clf = OneVsRestClassifier(lr)
clf.fit(X_train_multilabel, y_train_multilabel)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [70]:
prediction19 = clf.predict(X_test_multilabel)

precision19 = precision_score(y_test_multilabel, prediction19, average='micro')

recall19 = recall_score(y_test_multilabel, prediction19, average='micro')

f1_score19 = 2*((precision19 * recall19)/(precision19 + recall19))

print("precision19: {:.4f}, recall19: {:.4f}, F1-measure: {:.4f}".format(precision19, recall19, f1_score19))

precision19: 0.4180, recall19: 0.6732, F1-measure: 0.5158


In [71]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ",y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(prediction19[k])[0],"\n")

Movie:  Amityville: The Evil Escapes
Actual genre:  paranormal
Predicted tag:  ['cult' 'flashback' 'murder'] 

Movie:  Stitch! The Movie
Actual genre:  psychedelic
Predicted tag:  ['cult'] 

Movie:  36 Hours
Actual genre:  brainwashing, psychological
Predicted tag:  ['flashback'] 

Movie:  Blood on the Sun
Actual genre:  violence, murder
Predicted tag:  ['flashback' 'romantic'] 

Movie:  The Nightcomers
Actual genre:  gothic, murder, haunting
Predicted tag:  ['murder' 'romantic'] 



<h1>Conclusion</h1>

In [72]:
from prettytable import PrettyTable

tabel = PrettyTable()

tabel.field_names=['Model', 'Vectorizer', 'Precision','recall','f1_score']


tabel.add_row(['SGDClassifier(log)', 'AVG W2V',  round(precision17, 3), round(recall17, 3), round(f1_score17, 3)])

tabel.add_row(['SGDClassifier(hinge)','AVG W2V', round(precision18, 3),  round(recall18, 3), round(f1_score18, 3)])

tabel.add_row(['LogisticRegression','AVG W2V', round(precision19, 3), round(recall19, 3), round(f1_score19, 3)])


print(tabel)

+----------------------+------------+-----------+--------+----------+
|        Model         | Vectorizer | Precision | recall | f1_score |
+----------------------+------------+-----------+--------+----------+
|  SGDClassifier(log)  |  AVG W2V   |   0.403   | 0.654  |  0.499   |
| SGDClassifier(hinge) |  AVG W2V   |   0.436   | 0.646  |   0.52   |
|  LogisticRegression  |  AVG W2V   |   0.418   | 0.673  |  0.516   |
+----------------------+------------+-----------+--------+----------+


<h1>6. LSTM-CNN Model</h1>

In [73]:
max_review_length = 400
X_train = sequence.pad_sequences(X_train_multilabel, maxlen=max_review_length, padding='post')
X_test = sequence.pad_sequences(X_test_multilabel, maxlen=max_review_length, padding='post')

In [74]:
inputt = 8252
batch_size = 32
epochs = 10

In [75]:
model = Sequential()

model.add(Embedding(inputt, 50, input_length = max_review_length))

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(BatchNormalization())

model.add(LSTM(100, return_sequences=True))

model.add(Dropout(0.2))

model.add(Activation('relu'))

model.add(BatchNormalization())

model.add(GlobalMaxPooling1D())

model.add(Dense(5, activation='sigmoid'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           412600    
_________________________________________________________________
lstm_1 (LSTM)                (None, 400, 100)          60400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 400, 100)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 100)          400       
_________________________________________________________________
lstm_2 (LSTM)                (None, 400, 100)          80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 400, 100)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 400, 100)         

In [76]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [77]:
model.fit(X_train, y_train_multilabel, 
          batch_size = batch_size,
          validation_data=(X_test, y_test_multilabel),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 10989 samples, validate on 2768 samples
Epoch 1/10
10989/10989 [==============================] - 658s 60ms/step - loss: 2.0326 - accuracy: 0.1586 - val_loss: 2.2101 - val_accuracy: 0.0773
Epoch 2/10
10989/10989 [==============================] - 658s 60ms/step - loss: 2.0154 - accuracy: 0.1598 - val_loss: 2.0857 - val_accuracy: 0.0773
Epoch 3/10
10989/10989 [==============================] - 662s 60ms/step - loss: 2.0155 - accuracy: 0.0752 - val_loss: 2.4300 - val_accuracy: 0.0773
Epoch 4/10
10989/10989 [==============================] - 700s 64ms/step - loss: 2.0102 - accuracy: 0.1167 - val_loss: 2.0893 - val_accuracy: 0.0773
Epoch 5/10
10989/10989 [==============================] - 680s 62ms/step - loss: 2.0082 - accuracy: 0.2172 - val_loss: 2.0490 - val_accuracy: 0.2132
Epoch 6/10
10989/10989 [==============================] - 654s 60ms/step - loss: 2.0029 - accuracy: 0.2212 - va

In [78]:
test_loss, test_acc = model.evaluate(X_test, y_test_multilabel, verbose=2)

print('\nTest accuracy:', test_acc)


Test accuracy: 0.213150292634964


In [79]:
model.save('lstm_model_top5.h5') #Saving the Model for Future Use

In [80]:
model = load_model('lstm_model_top5.h5') #Loading the Model

In [81]:
model_prediction = model.predict(X_test, verbose=0)

In [82]:
for i in range(5):
    k = test.sample(1).index[0]
    print("Movie: ", test['title'][k]) 
    print("Actual genre: ", y_test[k])
    print("Predicted tag: ", cnt_vectorizer.inverse_transform(model_prediction[k])[0],"\n")

Movie:  Dawn of the Mummy
Actual genre:  violence, murder
Predicted tag:  ['cult' 'flashback' 'murder' 'romantic' 'violence'] 

Movie:  La lupa mannara
Actual genre:  paranormal, insanity, revenge, murder, violence
Predicted tag:  ['cult' 'flashback' 'murder' 'romantic' 'violence'] 

Movie:  Paris, je t'aime
Actual genre:  romantic, cult, storytelling, flashback
Predicted tag:  ['cult' 'flashback' 'murder' 'romantic' 'violence'] 

Movie:  Dead Poets Society
Actual genre:  dramatic, sentimental, inspiring, philosophical, melodrama
Predicted tag:  ['cult' 'flashback' 'murder' 'romantic' 'violence'] 

Movie:  Ida
Actual genre:  murder
Predicted tag:  ['cult' 'flashback' 'murder' 'romantic' 'violence'] 

